In [1]:
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import nest_asyncio
import ssl

In [2]:
# Set environment variables to disable SSL verification
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

# Create a custom SSL context that doesn't verify certificates
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Override the default SSL context
ssl._create_default_https_context = lambda: ssl_context

In [3]:
nest_asyncio.apply()

test_client = httpx.Client(verify=False, timeout=30.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

llm = ChatOpenAI(temperature=0.9, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=test_client)

def is_userprovided(service):
    return service["name"] == "time-mcp-server"
user_provided_services = filter(is_userprovided, services["user-provided"])
user_provided_credentials = list(user_provided_services)[0]["credentials"]

print(user_provided_credentials["mcpServiceURL"])
time_mcp = user_provided_credentials["mcpServiceURL"] + "/sse"

def is_petclinic_userprovided(service):
    return service["name"] == "petclinic-mcp-server"
user_provided_petclinic_services = filter(is_petclinic_userprovided, services["user-provided"])
user_provided_petclinic_credentials = list(user_provided_petclinic_services)[0]["credentials"]

print(user_provided_petclinic_credentials["mcpServiceURL"])
petclinic_mcp = user_provided_petclinic_credentials["mcpServiceURL"] + "/sse"

# Async HTTP client for MCP
#async_client = httpx.AsyncClient(http2=True, verify=False, timeout=10.0)
# Patch httpx globally for MCP client
original_async_client = httpx.AsyncClient

class NoSSLAsyncClient(httpx.AsyncClient):
    def __init__(self, **kwargs):
         kwargs['verify'] = False
         super().__init__(**kwargs)

httpx.AsyncClient = NoSSLAsyncClient

# Create MCP client
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "sse",
            "url": time_mcp
        },
        "petclinic": {
            "transport": "sse",
            "url": petclinic_mcp
        }
    }
)
#Fetch and print tools
tools = await client.get_tools()
print("MCP Tools:", tools)


https://time-mcp-server.apps.tas-ndc.kuhn-labs.com
https://petclinic-mcp.apps.tas-ndc.kuhn-labs.com
MCP Tools: [StructuredTool(name='getCurrentTime', description='Get the current time', args_schema={'type': 'object', 'properties': {}, 'required': [], 'additionalProperties': False}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f3e7b2c7d80>), StructuredTool(name='fetch_petsByStatus', description='Retrieve pets from petstore api', args_schema={'properties': {'status': {'title': 'Status', 'type': 'string'}}, 'required': ['status'], 'title': 'fetch_petsByStatusArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f3e7b2c6ac0>)]


In [4]:
# --- Main flow ---
async def main():
    # Create MCP client
    client = MultiServerMCPClient(
        {
            "time": {"transport": "sse", "url": time_mcp},
            "petclinic": {"transport": "sse", "url": petclinic_mcp}
        }
    )

    # Get tools
    tools = await client.get_tools()
    print("MCP Tools:", [t.name for t in tools])

    # --- Test tools directly ---
    time_tool = [t for t in tools if t.name == "getCurrentTime"][0]
    print("Time Tool Result:", await time_tool.arun({}))

    pet_tool = [t for t in tools if t.name == "fetch_petsByStatus"][0]
    print("Pet Tool Result:", await pet_tool.arun({"status": "available"}))

    # --- Initialize agent ---
    try:
        agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.OPENAI_FUNCTIONS,  # structured input support
            handle_parsing_errors=True,
            verbose=True
        )
        print("Agent initialized.")
    except Exception as e:
        print("Agent initialization failed:", e)
        return

    # --- Call agent ---
    result = await agent.ainvoke({"input": "whats the date today?"})
    print("Agent Result:", result)

# Run
asyncio.run(main())

MCP Tools: ['getCurrentTime', 'fetch_petsByStatus']
Time Tool Result: [2025,8,21,23,7,58,302718294]
Pet Tool Result: ['{\n  "id": 7,\n  "category": {\n    "id": 4,\n    "name": "Lions"\n  },\n  "name": "Lion 1",\n  "photoUrls": [\n    "url1",\n    "url2"\n  ],\n  "tags": [\n    {\n      "id": 1,\n      "name": "tag1"\n    },\n    {\n      "id": 2,\n      "name": "tag2"\n    }\n  ],\n  "status": "available"\n}', '{\n  "id": 9,\n  "category": {\n    "id": 4,\n    "name": "Lions"\n  },\n  "name": "Lion 3",\n  "photoUrls": [\n    "url1",\n    "url2"\n  ],\n  "tags": [\n    {\n      "id": 1,\n      "name": "tag3"\n    },\n    {\n      "id": 2,\n      "name": "tag4"\n    }\n  ],\n  "status": "available"\n}', '{\n  "id": 37109534,\n  "category": {\n    "id": 1,\n    "name": "Dogs"\n  },\n  "name": "doggie-1755734818563",\n  "photoUrls": [\n    "https://cdn.sanity.io/images/5vm5yn1d/pro/5cb1f9400891d9da5a4926d7814bd1b89127ecba-1300x867.jpg?fm=webp&q=80"\n  ],\n  "tags": [\n    {\n      "id": 0

/home/vcap/tmp/ipykernel_47363/2480847178.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


APIConnectionError: Connection error.

In [ ]:
# get current time
time_tool = [t for t in tools if t.name == "getCurrentTime"][0]
result = await time_tool.arun({})
print("Time tool result:", result)

# fetch pets
pet_tool = [t for t in tools if t.name == "fetch_petsByStatus"][0]
result = await pet_tool.arun({"status": "available"})
print("Pet tool result:", result)